In [ ]:
import os
from dotenv import load_dotenv
from smolagents import OpenAIServerModel, ToolCallingAgent, Tool
from typing import List, Dict, Union
from sqlalchemy import create_engine, text, inspect
import re
import requests
from sqlalchemy import (
    create_engine,
    inspect,
    text,
    exc,
    Engine 
)
from sqlalchemy.exc import SQLAlchemyError as exc
import decimal
import datetime

load_dotenv()

d:\Python\AI_agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

register()
SmolagentsInstrumentor().instrument()

OpenTelemetry Tracing Details
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



# Инициализация инструментов

## Конвертация валюты

In [3]:
class CurrencyConversionTool(Tool):
    """Инструмент для конвертации валют с использованием API exchangerate-api.com."""
    name = "currency_converter"
    description = "Используется для конвертации валюты и получения актуальных курсов валют. Конвертирует указанную сумму из базовой валюты в целевую."
    inputs = {
        "base_currency": {"type": "string", "description": "Базовая валюта (например, 'USD')."},
        "target_currency": {"type": "string", "description": "Целевая валюта (например, 'EUR')."},
        "amount": {"type": "number", "description": "Сумма для конвертации. По умолчанию 1.0.", "nullable": True}
    }
    output_type = "object"

    def __init__(self, api_key: str):
        """Инициализирует инструмент с ключом API.

        Args:
            api_key: Ключ API для exchangerate-api.com.
        """
        super().__init__()
        if not api_key:
            raise ValueError("Необходимо предоставить ключ API для CurrencyConversionTool")
        self.api_key = api_key

    def forward(self, base_currency: str, target_currency: str, amount: float = 1.0) -> Dict[str, float]: 
        """Выполняет конвертацию валюты.

        Args:
            base_currency: Код валюты для конвертации. Например, 'USD'.
            target_currency: Код валюты, в которую нужно конвертировать. Например, 'EUR'.
            amount: Сумма для конвертации. По умолчанию 1.0.

        Returns:
            Tuple[float, float]: Кортеж, содержащий (conversion_rate, conversion_result).
            conversion_rate - обменный курс между валютами.
            conversion_result - сконвертированная сумма в целевой валюте.
        """
        endpoint = f"https://v6.exchangerate-api.com/v6/{self.api_key}/latest/{base_currency.upper()}"

        try:
            response = requests.get(endpoint, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            if data.get("result") != "success":
                raise ValueError(f"API Error: {data.get('error-type', 'Unknown')}")
            
            rates = data["conversion_rates"]
            if target_currency.upper() not in rates:
                raise ValueError(f"Валюта {target_currency} не найдена")
            
            rate = rates[target_currency.upper()]
            return {
                "conversion_rate": rate,
                "conversion_result": amount * rate
            }
            
        except requests.RequestException as e:
            raise ConnectionError(f"Ошибка запроса: {str(e)}")

## Обращение к базе данных

In [ ]:
engine = create_engine("sqlite:///user_transactions.db") 
inspector = inspect(engine)

In [7]:
class ListTablesTool(Tool):
    name = "list_tables"
    description = "Возвращает структуру таблиц с примерами данных и уникальными значениями столбцов"
    inputs: dict = {}
    output_type: str = "array"

    def __init__(self, db_url: str = "sqlite:///user_transactions.db"):
        super().__init__()
        self.engine = create_engine(db_url)
        self.inspector = inspect(self.engine)

    def _get_column_samples(self, table: str, column: str, col_type: str) -> Dict:
        """Возвращает метаданные для столбца"""
        try:
            with self.engine.connect() as conn:
                query = text(
                    f"SELECT DISTINCT {column} FROM {table} "
                    f"WHERE {column} IS NOT NULL LIMIT 1000"
                )
                result = conn.execute(query)
                values = [row[0] for row in result.fetchall()]

                # Форматирование значений
                samples = []
                for v in values[:5]:
                    if isinstance(v, (int, float)):
                        samples.append(f"{v:.2f}" if isinstance(v, float) else str(v))
                    else:
                        samples.append(str(v)[:50])

                # Анализ уникальных значений
                unique_count = len(values)
                metadata = {
                    "name": column,
                    "type": col_type,
                    "unique_values": unique_count,
                    "examples": samples if unique_count > 0 else ["NULL"]
                }

                #Специальная обработка для валют
                if column.lower() == "currency":
                    metadata["allowed_values"] = ["USD", "RUB", "EUR"]
                
                return metadata

        except SQLAlchemyError as e:
            return {
                "name": column,
                "type": col_type,
                "error": str(e)
            }

    def forward(self) -> List[Dict]:
        """Возвращает расширенную структуру таблиц"""

        tables_meta = []
        table_names = self.inspector.get_table_names()
        
        for table in table_names:
            # Получаем оригинальный DDL
            with self.engine.connect() as conn:
                result = conn.execute(
                    text(f"SELECT sql FROM sqlite_schema WHERE type='table' AND name='{table}';")
                )
                create_statement = result.scalar()

            # Собираем метаданные столбцов
            columns_meta = []
            for col in self.inspector.get_columns(table):
                col_meta = self._get_column_samples(table, col['name'], str(col['type']))
                columns_meta.append(col_meta)

            # Формируем комментарии для DDL
            comments = []
            for col in columns_meta:
                comment = f"/* {col['name']}: "
                if "allowed_values" in col:
                    comment += f"Allowed values: {', '.join(col['allowed_values'])}. "
                comment += f"Примеры: {', '.join(col['examples'])} */"
                comments.append(comment)
                
            tables_meta.append({
                "table_name": table,
                "ddl": create_statement,
                "columns": columns_meta,
                "ddl_with_comments": f"{create_statement}\n" + "\n".join(comments)
            })

        return tables_meta


In [ ]:
class ExecuteQueryTool(Tool):
    name = "execute_query"
    description = """
    Безопасно выполняет SQL-запросы SELECT к финансовой базе данных.
    """
    inputs = {
        "query": {
            "type": "string", 
            "description": "SQL-запрос SELECT. Примеры: "
                           "1. SELECT currency, SUM(amount) FROM transactions WHERE operation_type = 'income' GROUP BY currency "
                           "2. SELECT * FROM transactions WHERE location = 'Diaz PLC' AND operation_date > '2025-01-01'"
        }
    }
    output_type = "array"

    def __init__(self, engine: Engine):
        super().__init__()
        self.engine = engine

    def forward(self, query: str) -> List[Dict]:
        """Выполняет SQL-запрос с валидацией и обработкой ошибок"""
        try:
            self._validate_query(query)
            
            with self.engine.connect() as conn:
                result = conn.execute(text(query).execution_options(autocommit=True))
                
                if not result.returns_rows:
                    return [{"message": "Запрос успешно выполнен (нет результатов)"}]

                columns = result.keys()
                return [
                    {col: self._convert_value(row[col]) for col in columns}
                    for row in result.mappings()
                ]
                
        except exc as e:
            return [{"error": "Ошибка базы данных", "details": str(e)}]
        except Exception as e:
            return [{"error": "Внутренняя ошибка", "details": str(e)}]

    def _convert_value(self, value):
        """Конвертирует специальные типы данных"""
        if isinstance(value, decimal.Decimal):
            return float(value)
        if isinstance(value, datetime.date):
            return value.isoformat() 
        return value

    def _validate_query(self, query: str):
        """Проверяет запрос на безопасность"""
        query = query.upper().strip()
        
        # Проверка типа запроса
        if not query.startswith("SELECT"):
            raise ValueError("Разрешены только SELECT-запросы")

        # Запрещенные операции
        forbidden_keywords = {
            "INSERT", "UPDATE", "DELETE", "DROP", 
            "ALTER", "CREATE", "TRUNCATE", "GRANT"
        }
        for keyword in forbidden_keywords:
            if keyword in query:
                raise ValueError(f"Запрещенная операция: {keyword}")

        # Проверка доступных таблиц
        allowed_tables = {"TRANSACTIONS", "CURRENCIES"} 
        from_match = re.search(r"FROM\s+(\w+)", query, re.IGNORECASE)
        if from_match and from_match.group(1).upper() not in allowed_tables:
            raise ValueError("Доступ к этой таблице запрещен")

## Калькулятор

In [ ]:
class CalculatorTool(Tool):
    name = "calculator"
    description = """
    Выполняет математические вычисления. Поддерживает:
    - Базовые арифметические операции (+-*/)
    - Суммирование списка чисел
    - Работу с десятичными числами
    """
    inputs = {
        "expression": {
            "type": "string",
            "description": "Выражение для вычисления. Примеры: "
                          "'45.7 + 128.91', "
                          "'сумма 100 200 300'"
        }
    }
    output_type = "float"

    def forward(self, expression: str) -> Dict[str, Union[float, str]]:
        try:
            # Нормализация выражения
            expr = expression.lower().replace(',', '.').strip()
            
            # Обработка команды "сумма"
            if expr.startswith("сумма"):
                numbers = [float(n) for n in re.findall(r'\d+\.?\d*', expr)]
                result = sum(numbers)
            else:
                # Проверка на безопасные символы
                if not re.fullmatch(r'^[\d\s\.\+\-\*\/\(\)]+$', expr):
                    raise ValueError("Выражение содержит недопустимые символы")
                
                # Замена альтернативных символов операций
                expr = expr.replace('×', '*').replace('÷', '/')
                
                # Безопасное вычисление
                result = eval(expr, {'__builtins__': None}, {})
            
            return {"result": round(float(result), 2)}
            
        except Exception as e:
            return {"error": f"Ошибка вычисления: {str(e)}"}

# Инициализация модели

In [113]:
model = OpenAIServerModel(
    model_id="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
    api_base="https://api.together.xyz/v1/",
    api_key=os.environ["TOGETHER_API_KEY"],
    temperature=0.0,
)

In [114]:
currency_api_key = os.environ["currency_api_key"]

In [ ]:
agent_MLE = ToolCallingAgent(
    tools = [
        ListTablesTool(),
        CurrencyConversionTool(currency_api_key),
        ExecuteQueryTool(engine),
        CalculatorTool(),
        ],
    model = model,
)

In [126]:
Financial_Agent_Prompt = '''
Ты — интеллектуальный финансовый ассистент, специализирующийся на работе с базами данных и валютными операциями. Твоя задача — точно и эффективно решать финансовые запросы, используя доступные инструменты.

### Принципы работы:
1. **Последовательность действий**: Действуй пошагово, используя цикл "Action → Observation"
2. **Точность данных**: Всегда проверяй структуру данных перед запросами
3. **Если запрос общий** : Возвращай final_answer

### Строгие правила форматирования ответов:
1. ВСЕ ответы должны быть в формате VALID JSON
2. Никаких комментариев, пояснений или текста вне JSON
3. При размышлениях не пиши конкретные вызовы инструментов

### Доступные инструменты:
{%- for tool in tools.values() %}
- {{ tool.name }}: {{ tool.description }}
    Принимаемые входы: {{tool.inputs}}
    Типы возвращаемых данных: {{tool.output_type}}
{%- endfor %}


### Правила выполнения:
1. **Обязательность действий**: Каждый шаг должен заканчиваться вызовом инструмента
2. **Проверка данных**: Всегда начинай с list_tables при работе с новыми запросами, связанными с базой данных
3. **Оптимальные запросы**: Формируй SQL-запросы, которые:
   - Выбирают только нужные поля
   - Содержат условия WHERE для фильтрации
   - Используют агрегатные функции при необходимости
4. **Обработка ошибок**: При получении ошибки анализируй её и корректируй запрос
5. Чтобы выдать окончательный ответ на задачу, используй JSON-блок с инструментом "name": "final_answer". Это единственный способ завершить выполнение задачи — иначе ты застрянешь в бесконечном цикле. Твой финальный вывод должен выглядеть так:
  Action:
  {
    "name": "final_answer",
    "arguments": {"answer": "вставь здесь свой окончательный ответ"}
  }

### Примеры работы:

---
Пример 1: Анализ расходов
Задача: «Сколько всего денег я потратил первого январе 2025 года в рублях?»

Action:
{
  "name": "list_tables",
  "arguments": {}
}
Observation: {'table_name': 'transactions', 'ddl': 'CREATE TABLE transactions (\n        id INTEGER PRIMARY KEY 
AUTOINCREMENT,\n        currency TEXT,\n        amount REAL,\n        operation_type TEXT,\n        location 
TEXT,\n        comment TEXT,\n        operation_date TEXT\n    )', 'columns': |{'name': 'id', 'type': 'INTEGER', 
'unique_values': 1000, 'examples': |'1', '2', '3', '4', '5']}, {'name': 'currency', 'type': 'TEXT', 
'unique_values': 2, 'examples': |'USD', 'RUB'], 'allowed_values': |'USD', 'RUB', 'EUR']}, {'name': 'amount', 
'type': 'REAL', 'unique_values': 1000, 'examples': |'2907.07', '5333.08', '4579.34', '3387.71', '3628.03']}, 
{'name': 'operation_type', 'type': 'TEXT', 'unique_values': 2, 'examples': |'income', 'expense']}, {'name': 
'location', 'type': 'TEXT', 'unique_values': 1000, 'examples': |'Fry, Morales and Owens', 'Young-Jones', 'Miller 
Ltd', 'Larson and Sons', 'Banks Group']}, {'name': 'comment', 'type': 'TEXT', 'unique_values': 1000, 'examples': 
|'Marriage somebody begin.', 'Such control challenge make.', 'Community dinner successful.', 'Can.', 'Cup form 
generation.']}, {'name': 'operation_date', 'type': 'TEXT', 'unique_values': 395, 'examples': |'2024-11-29', 
'2024-12-26', '2024-11-23', '2025-02-23', '2025-03-25']}]

Action:
{
  "name": "execute_query",
  "arguments": {
    "query": "SELECT amount, currency FROM transactions WHERE operation_type = 'expense' AND operation_date = '2025-01-01'"
  }
}
Observation: [{'amount': 3482.12, 'currency': 'RUB'}, {'amount': 5429.09, 'currency': 'USD'}, {'amount': 6619.26, 
'currency': 'USD'}, {'amount': 1072.47, 'currency': 'USD'}, {'amount': 3264.92, 'currency': 'RUB'}, {'amount': 
7980.51, 'currency': 'USD'}, {'amount': 9867.05, 'currency': 'USD'}, {'amount': 3810.68, 'currency': 'USD'}, 
{'amount': 474.57, 'currency': 'USD'}, {'amount': 1699.91, 'currency': 'USD'}, {'amount': 7845.17, 'currency': 
'USD'}, {'amount': 1873.52, 'currency': 'USD'}, {'amount': 3170.8, 'currency': 'USD'}]

Action:
{
  "name": "currency_converter",
  "arguments": {
    "base_currency": "USD",
    "target_currency": "RUB",
    "amount": 1500
  }
}
Observation: {"amount": 1620.75, "rate": 1.0805}

Action:
{
  "name": "calculator",
  "arguments": { 5429.09 * 1.0805 + 3482.12 + 3810.68 * 1.0805
  }
}
Observation: {"amount": 10054.09}

Action:
{
  "name": "final_answer",
  "arguments": {
    "answer": "Траты за 1 января 2025 года составили 10054.09 рублей"
  }
}

---
Пример 2: Конвертация валют
Задача: «Сколько будет 1500 EUR в USD по текущему курсу?»

Action:
{
  "name": "currency_converter",
  "arguments": {
    "base_currency": "EUR",
    "target_currency": "USD",
    "amount": 1500
  }
}
Observation: {"amount": 1620.75, "rate": 1.0805}

Action:
{
  "name": "final_answer",
  "arguments": {
    "answer": "1500 EUR = 1620.75 USD (курс 1.0805)"
  }
}

---
Пример 3: Получение данных о базе данных
Задача: «Какие таблицы содержатся в базе данных»

Action:
{
  "name": "list_tables",
  "arguments": {}
}
Observation: {'table_name': 'transactions', 'ddl': 'CREATE TABLE transactions (\n        id INTEGER PRIMARY KEY 
AUTOINCREMENT,\n        currency TEXT,\n        amount REAL,\n        operation_type TEXT,\n        location 
TEXT,\n        comment TEXT,\n        operation_date TEXT\n    )', 'columns': |{'name': 'id', 'type': 'INTEGER', 
'unique_values': 1000, 'examples': |'1', '2', '3', '4', '5']}, {'name': 'currency', 'type': 'TEXT', 
'unique_values': 2, 'examples': |'USD', 'RUB'], 'allowed_values': |'USD', 'RUB', 'EUR']}, {'name': 'amount', 
'type': 'REAL', 'unique_values': 1000, 'examples': |'2907.07', '5333.08', '4579.34', '3387.71', '3628.03']}, 
{'name': 'operation_type', 'type': 'TEXT', 'unique_values': 2, 'examples': |'income', 'expense']}, {'name': 
'location', 'type': 'TEXT', 'unique_values': 1000, 'examples': |'Fry, Morales and Owens', 'Young-Jones', 'Miller 
Ltd', 'Larson and Sons', 'Banks Group']}, {'name': 'comment', 'type': 'TEXT', 'unique_values': 1000, 'examples': 
|'Marriage somebody begin.', 'Such control challenge make.', 'Community dinner successful.', 'Can.', 'Cup form 
generation.']}, {'name': 'operation_date', 'type': 'TEXT', 'unique_values': 395, 'examples': |'2024-11-29', 
'2024-12-26', '2024-11-23', '2025-02-23', '2025-03-25']}

Action:
{
  "name": "final_answer",
  "arguments": {
    "answer": "В базе данных присутствует таблица transactions c названиями колонок : "id", "amount", "currency", "operation_type", "location", "commet", "operation_date""
  }
}

### Критические требования:
1. Никогда не изменяй базу данных (только SELECT)
2. Все суммы в ответах должны указывать валюту
3. Для дат используй формат YYYY-MM-DD
4. При работе с периодами всегда проверяй наличие данных
5. Если запрос требует нескольких шагов - сохраняй промежуточные результаты
6. Все вычисления выполняй через calculator
7. Ответ через `final_answer` должен включать не только итоговую строку, но и краткое обоснование каждого шага, ссылки на использованные инструменты и пояснение формул или SQL-конструкций.

Твоя цель — предоставлять точные, проверяемые финансовые данные с минимальным количеством запросов.'''

In [ ]:
agent_MLE.prompt_templates['system_prompt'] = Financial_Agent_Prompt

In [128]:
print(agent_MLE.system_prompt)


Ты — интеллектуальный финансовый ассистент, специализирующийся на работе с базами данных и валютными операциями. Твоя задача — точно и эффективно решать финансовые запросы, используя доступные инструменты.

### Принципы работы:
1. **Последовательность действий**: Действуй пошагово, используя цикл "Action → Observation"
2. **Точность данных**: Всегда проверяй структуру данных перед запросами
3. **Если запрос общий** : Возвращай final_answer

### Строгие правила форматирования ответов:
1. ВСЕ ответы должны быть в формате VALID JSON
2. Никаких комментариев, пояснений или текста вне JSON
3. При размышлениях не пиши конкретные вызовы инструментов

### Инструкции по вычислениям:
1. Все вычисления выполняй через calculator
2. Для суммирования используй формат: "сумма 100 200 300"
3. Для сложных выражений: "45.7 + 128.91 * 0.2"

### Доступные инструменты:
- list_tables: Возвращает структуру таблиц с примерами данных и уникальными значениями столбцов
    Принимаемые входы: {}
    Типы возвраща

In [131]:
task = ''' Сколько всего денег я потратил первого январе 2025 года. Учти все расходы и переведи к одной валюте, а именно к рублям?
'''

In [132]:
analyst_result = agent_MLE.run(
    task
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Сколько всего денег я потратил первого январе 2025 года. Учти все расходы и переведи к одной валюте, а именно к │
│ рублям?                                                                                                         │
│                                                                                                                 │
╰─ OpenAIServerModel - deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free ────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'list_tables' with arguments: {}                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |{'table_name': 'transactions', 'ddl': 'CREATE TABLE transactions (\n        id INTEGER PRIMARY KEY 
AUTOINCREMENT,\n        currency TEXT,\n        amount REAL,\n        operation_type TEXT,\n        location 
TEXT,\n        comment TEXT,\n        operation_date TEXT\n    )', 'columns': |{'name': 'id', 'type': 'INTEGER', 
'unique_values': 1000, 'examples': |'1', '2', '3', '4', '5']}, {'name': 'currency', 'type': 'TEXT', 
'unique_values': 2, 'examples': |'USD', 'RUB'], 'allowed_values': |'USD', 'RUB', 'EUR']}, {'name': 'amount', 
'type': 'REAL', 'unique_values': 1000, 'examples': |'2907.07', '5333.08', '4579.34', '3387.71', '3628.03']}, 
{'name': 'operation_type', 'type': 'TEXT', 'unique_values': 2, 'examples': |'income', 'expense']}, {'name': 
'location', 'type': 'TEXT', 'unique_values': 1000, 'examples': |'Fry, Morales and Owens', 'Young-Jones', 'Miller 
Ltd', 'Larson and Sons', 'Banks Group']}, {'name': 'comment', 'type': 'TEXT', 'unique_values': 1000, 'examples': 
|'Marriage somebody begin.', 'Such control challenge make.', 'Community dinner successful.', 'Can.', 'Cup form 
generation.']}, {'name': 'operation_date', 'type': 'TEXT', 'unique_values': 395, 'examples': |'2024-11-29', 
'2024-12-26', '2024-11-23', '2025-02-23', '2025-03-25']}], 'ddl_with_comments': 'CREATE TABLE transactions (\n     
id INTEGER PRIMARY KEY AUTOINCREMENT,\n        currency TEXT,\n        amount REAL,\n        operation_type TEXT,\n
location TEXT,\n        comment TEXT,\n        operation_date TEXT\n    )\n/* id: Примеры: 1, 2, 3, 4, 5 */\n/* 
currency: Allowed values: USD, RUB, EUR. Примеры: USD, RUB */\n/* amount: Примеры: 2907.07, 5333.08, 4579.34, 
3387.71, 3628.03 */\n/* operation_type: Примеры: income, expense */\n/* location: Примеры: Fry, Morales and Owens, 
Young-Jones, Miller Ltd, Larson and Sons, Banks Group */\n/* comment: Примеры: Marriage somebody begin., Such 
control challenge make., Community dinner successful., Can., Cup form generation. */\n/* operation_date: Примеры: 
2024-11-29, 2024-12-26, 2024-11-23, 2025-02-23, 2025-03-25 */'}]

[Step 1: Duration 9.19 seconds| Input tokens: 2,741 | Output tokens: 416]

...

[Step 4: Duration 10.79 seconds| Input tokens: 13,926 | Output tokens: 1,771]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Общая сумма трат за 1 января 2025 года составляет      │
│ 4,131,073.35 рублей. Это включает все расходы, переведенные в рубли по текущему курсу 82.7463.'}                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Общая сумма трат за 1 января 2025 года составляет 4,131,073.35 рублей. Это включает все расходы, 
переведенные в рубли по текущему курсу 82.7463.

[Step 5: Duration 9.65 seconds| Input tokens: 18,240 | Output tokens: 2,227]